# Batch Generation

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaForCausalLM

# model_id = 'huggyllama/llama-7b'
model_id = 'huggyllama/llama-65b'
# model_id = 'meta-llama/Llama-2-7b-hf'
# model_id = 'meta-llama/Llama-2-13b-hf'
# model_id = 'meta-llama/Llama-2-70b-hf'
# model_id = 'epfl-llm/meditron-70b'
USE_FINETUNED = False
N_SHOT = 0
# DATASET = 'SNUH_test'
DATASET = 'CDDCESM'
# DATASET = 'SNUH_all'
BATCH_SZ = 2
DEVICE = 0
lora_r= 2

In [2]:
IS_ZEROSHOT = 1 if not USE_FINETUNED and N_SHOT==0 else 0

In [3]:
#Load from checkpoint
# checkpoint_name = '/lora-alpaca-MG-30b/checkpoint-50/pytorch_model.bin' ## 데이터 100개 훈련 모델
# checkpoint_name = '/home/workspace/lora-alpaca-MG-30b-1+2/checkpoint-80/pytorch_model.bin' ## 데이터 200개 훈련 모델
# checkpoint_name = '/home/workspace/lora-alpaca-MG-30b-364/checkpoint-80/pytorch_model.bin' ## 데이터 364개 훈련 모델
# checkpoint_name = '/home/workspace/lora-alpaca-MG-30b-500/checkpoint-116/pytorch_model.bin' ## 데이터 500개 훈련 모델
# checkpoint_name = '/home/workspace/lora_alpaca-MG-65b-500/checkpoint-116/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/lora_alpaca-MG-7b-500/checkpoint-156/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/lora_alpaca-MG-7b-500-r4/checkpoint-140/pytorch_model.bin' ## 
#==========================================================================================================================
### v1
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v1_7b_r8/checkpoint-148/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v1_30b_r8/checkpoint-44/pytorch_model.bin' ## 
#==========================================================================================================================
### v2
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2_7b_r8/checkpoint-124/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.1_7b_r8/checkpoint-108/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_7b_r8/checkpoint-144/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_30b_r8/checkpoint-116/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_30b_r2/checkpoint-116/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_30b_r16/checkpoint-92/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_30b_r4kqvo/checkpoint-104/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_30b_r1kqvo/checkpoint-164/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_13b_r2/checkpoint-84/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_13b_r8/checkpoint-88/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v2.2_13b_r1kqvo/checkpoint-144/pytorch_model.bin' ## 
#==========================================================================================================================
### v3
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_13b_r1kqvo/checkpoint-132/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_30b_r1kqvo/checkpoint-132/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_7b_r8/checkpoint-172/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_13b_r2/checkpoint-240/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_30b_r4kqvo/checkpoint-168/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_13b_r8/checkpoint-156/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_30b_r16/checkpoint-72/pytorch_model.bin' ## 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_30b_r8/checkpoint-64/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_30b_r2/checkpoint-212/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.0_30b_r2/checkpoint-156/pytorch_model.bin' 
#==========================================================================================================================
### v3.2
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r2/checkpoint-284/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r2/checkpoint-212/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r2/checkpoint-100/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r2/checkpoint-24/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r1/checkpoint-144/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r1_cutoff512/checkpoint-144/pytorch_model.bin' 

# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r1all_bz8/checkpoint-488/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r1all_bz8/checkpoint-580/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r2all_bz8/checkpoint-580/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_30b_r4all_bz8/checkpoint-580/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_7b_r2all_bs8/checkpoint-476/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_65b_r2all_bs8/checkpoint-580/pytorch_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_13b_r2all_bs8/checkpoint-496/pytorch_model.bin' 

# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_Llama2_7b_r2all_bs8/checkpoint-420/adapter_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_Llama2_7b_r2all_bs8/adapter_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_7b_r2all_bs8_again2/checkpoint-420/adapter_model.bin' 

# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_70b_r2all_bs8_again/adapter_model.bin' 
# checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_Llama2_70b_r2all_bs8/adapter_model.bin'
checkpoint_name = '/home/workspace/MGLabler_trained_model/v3.2_Llama2_13b_r2all_bs8/adapter_model.bin'



In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"  # Allow batched inference

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = LlamaForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config, 
    # device_map='auto',
    device_map={"":DEVICE},
    cache_dir='/home/public_models'
)

Loading checkpoint shards: 100%|██████████| 14/14 [00:42<00:00,  3.06s/it]


In [5]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    # cls = bnb.nn.Linear4bit if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [6]:
if USE_FINETUNED:
    from peft import (
        LoraConfig,
        get_peft_model,
        get_peft_model_state_dict,
        prepare_model_for_kbit_training,
        set_peft_model_state_dict,
    )
    lora_target_modules= find_all_linear_names(model)
    # lora_target_modules= [
    #     "q_proj",
    #     "v_proj",
    #     # "k_proj",
    #     # "o_proj",
    # ]

    config = LoraConfig(
        r=lora_r,
        lora_alpha=16,
        target_modules=lora_target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config)
    set_peft_model_state_dict(model, torch.load(checkpoint_name, map_location=torch.device(f"cuda:{DEVICE}")))

In [7]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [8]:
# #Load from saved
# from peft import PeftModel
# model = PeftModel.from_pretrained(model, '/lora-alpaca-MG-30b/')

In [33]:
import pandas as pd

if DATASET == 'SNUH_test':
    data_path = '/home/data/SNUH_Mammo/0518/MGLabler_test_data_v5.xlsx'
    data = pd.read_excel(data_path)
elif DATASET == 'CDDCESM':
    data_path = '/home/data/CDD-CESM/CDD-CESM_reports_w_label_wo_CESM_fixed1.3.xlsx'
    data = pd.read_excel(data_path)
elif DATASET == 'SNUH_train':
    data_path = '/home/data/SNUH_Mammo/0518/MGLabler_training_data_labeled_v2.2.xlsx'
    data = pd.read_excel(data_path)

elif DATASET == 'SNUH_all':
    data_path = '/home/data/SNUH_Mammo/0518/mammo_screening0531_w_processedFinding_prior.csv'
    data = pd.read_csv(data_path)

version = data_path.split('_')[-1][:-5]
print(DATASET, version)

CDDCESM fixed1.3


In [10]:
# import pandas as pd
# # data = pd.read_csv('/home/data/SNUH_Mammo/0518/mammo_screening0531_w_processedFinding_prior.csv')
# data = pd.read_excel('/home/data/SNUH_Mammo/0518/MGLabler_training_data_labeled_v1.xlsx')

In [11]:
if not USE_FINETUNED:
    import json
    import random
    random.seed(1125)

    train_path = '/home/data/SNUH_Mammo/0518/MG_training_data_labeled_v3.2_w_inst.json'
    with open(train_path, "r", encoding="utf-8-sig") as f:
        train = json.load(f)

In [34]:
instruction = open('/home/workspace/lora_finetune/alpaca-lora/mg_labeler_instruction_prompt.txt', 'r').read()
bars='-'*10
if USE_FINETUNED:
    prompt = [f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input: {f}\n\n### Response:" for f in data['processed_finding']]
elif N_SHOT>0: 
    instruction = instruction[:-1] + f" like {N_SHOT} examples below."

    prompt = []
    for f in data['processed_finding']:
        eg = random.sample(train, k=N_SHOT)
        examples = ""
        for k in range(N_SHOT):
            text = f"""Example{k+1}: ### Input: {eg[k]['input']}\n### Response: {eg[k]['output']}"""
            examples += text + "\n"

        # pt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n{examples}\n\n### Input: {f}\n### Response:"""
        pt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:{instruction}\n\n{bars}\n{examples}{bars}\n\nHere is the report that I want you to extract lesion information.\n### Input: {f}\n### Response:"""

        prompt.append(pt)
else:
    # zero-shot prompt
    prompt = []
    for f in data['processed_finding']:      
        pt = """As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. 

Requirements:
1. Provide the lesion's location as 'right', 'left' or 'both'(i.e, right and left) if the lesion is explicitly mentioned as present.
2. Provide 'obscure' if the information is incomplete.
3. Default to 'x' if the finding is not mentioned.

Template:
{
'Nodule': '[ANSWER]',
'Mass': '[ANSWER]',
'Calcification': '[ANSWER]',
'Asymmetry': '[ANSWER]',
'Architectural Distortion': '[ANSWER]',
'Skin Thickening': '[ANSWER]',
'Lymph Node Enlargement': '[ANSWER]',
'Intramammary Lymph Node': '[ANSWER]',
'Nipple Retraction': '[ANSWER]',
'Skin Retraction': '[ANSWER]',
'Trabecular Thickening': '[ANSWER]',
}

Report:
""" + f + """

Answer according to the template:
{
'Nodule': '"""
        prompt.append(pt)

In [13]:
def post_process_llama_label(text, is_zeroshot=False):
    if is_zeroshot == False:
        text = text.split('### Response:')[-1]
    elif is_zeroshot:
        text = text.split('Answer according to the template:')[-1]
        text = text.split("}")[0]+'}'
        text = text.replace('\n','')
        
    text = text.replace('right','r')
    text = text.replace('left', 'l')
    text = text.replace('both', 'b')
    text = text.replace('obscure', 'o')
    return text.strip()

In [35]:
print(prompt[0])

As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. 

Requirements:
1. Provide the lesion's location as 'right', 'left' or 'both'(i.e, right and left) if the lesion is explicitly mentioned as present.
2. Provide 'obscure' if the information is incomplete.
3. Default to 'x' if the finding is not mentioned.

Template:
{
'Nodule': '[ANSWER]',
'Mass': '[ANSWER]',
'Calcification': '[ANSWER]',
'Asymmetry': '[ANSWER]',
'Architectural Distortion': '[ANSWER]',
'Skin Thickening': '[ANSWER]',
'Lymph Node Enlargement': '[ANSWER]',
'Intramammary Lymph Node': '[ANSWER]',
'Nipple Retraction': '[ANSWER]',
'Skin Retraction': '[ANSWER]',
'Trabecular Thickening': '[ANSWER]',
}

Report:
Left Breast:  Upper architectural distortion is seen.  No suspicious microcalcifications.  Normal skin thickness and contour of breast.      OPINION:  Left Breast:  Upper architectural distortion (BIR

In [36]:
default_dict={'N': 'o',
 'M': 'o',
 'C': 'o',
 'A': 'o',
 'AD': 'o',
 'ST': 'o',
 'LNE': 'o',
 'ILN': 'o',
 'NR': 'o',
 'SR': 'o',
 'TT': 'o'}
def generate_label(model, batch, is_zeroshot=False):
    max_new_tokens = 70
    if is_zeroshot:
        max_new_tokens = 140
    with torch.no_grad():
        generated_ids = model.generate(
                            **tokenizer(batch, 
                                return_tensors='pt', 
                                return_token_type_ids=False, 
                                truncation=False, 
                                padding=True, 
                                # max_length=2048
                                # max_length=512
                            ).to('cuda'),
                            # max_new_tokens=2048,
                            max_new_tokens=max_new_tokens,
                            early_stopping=True,
                            do_sample=False,
                            eos_token_id=2,
                            )
        
    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print(generated_texts)
    generated_dict = []
    for t in generated_texts:
        try:
            generated_label = eval(post_process_llama_label(t, is_zeroshot))
        except:
            generated_label = default_dict

        if type(generated_label) == dict:
            generated_dict.append(generated_label)
        else:
            generated_dict.append(default_dict)

    # generated_dict = [eval(post_process_llama_label(t))for t in generated_texts]
    return generated_dict    

In [37]:
def divide_batch(l:list, batch_size):
    for i in range(0, len(l), batch_size): 
        yield l[i:i + batch_size]

In [38]:
batchs = list(divide_batch(prompt, batch_size=BATCH_SZ)) ## 왜 제일 앞  29개 뻈지? 

In [39]:
from tqdm import tqdm
labels = []
for batch in tqdm(batchs):
    label = generate_label(model, batch, IS_ZEROSHOT)
    labels.append(label)
    
labels = sum(labels, [])

  0%|          | 0/163 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
  1%|          | 1/163 [02:14<6:04:03, 134.84s/it]

["As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \n\nRequirements:\n1. Provide the lesion's location as 'right', 'left' or 'both'(i.e, right and left) if the lesion is explicitly mentioned as present.\n2. Provide 'obscure' if the information is incomplete.\n3. Default to 'x' if the finding is not mentioned.\n\nTemplate:\n{\n'Nodule': '[ANSWER]',\n'Mass': '[ANSWER]',\n'Calcification': '[ANSWER]',\n'Asymmetry': '[ANSWER]',\n'Architectural Distortion': '[ANSWER]',\n'Skin Thickening': '[ANSWER]',\n'Lymph Node Enlargement': '[ANSWER]',\n'Intramammary Lymph Node': '[ANSWER]',\n'Nipple Retraction': '[ANSWER]',\n'Skin Retraction': '[ANSWER]',\n'Trabecular Thickening': '[ANSWER]',\n}\n\nReport:\nLeft Breast:  Upper architectural distortion is seen.  No suspicious microcalcifications.  Normal skin thickness and contour of breast.      OPINION:  Left Breast:  Upper arch

  1%|          | 2/163 [04:29<6:01:39, 134.78s/it]

["As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \n\nRequirements:\n1. Provide the lesion's location as 'right', 'left' or 'both'(i.e, right and left) if the lesion is explicitly mentioned as present.\n2. Provide 'obscure' if the information is incomplete.\n3. Default to 'x' if the finding is not mentioned.\n\nTemplate:\n{\n'Nodule': '[ANSWER]',\n'Mass': '[ANSWER]',\n'Calcification': '[ANSWER]',\n'Asymmetry': '[ANSWER]',\n'Architectural Distortion': '[ANSWER]',\n'Skin Thickening': '[ANSWER]',\n'Lymph Node Enlargement': '[ANSWER]',\n'Intramammary Lymph Node': '[ANSWER]',\n'Nipple Retraction': '[ANSWER]',\n'Skin Retraction': '[ANSWER]',\n'Trabecular Thickening': '[ANSWER]',\n}\n\nReport:\nRight Breast:  No speculated mass lesions or suspicious microcalcifications.  Retroareolar increased density that is tubular in shape likely dilated duct.  Normal skin thickne

  2%|▏         | 3/163 [06:44<5:59:11, 134.70s/it]

...

["As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \n\nRequirements:\n1. Provide the lesion's location as 'right', 'left' or 'both'(i.e, right and left) if the lesion is explicitly mentioned as present.\n2. Provide 'obscure' if the information is incomplete.\n3. Default to 'x' if the finding is not mentioned.\n\nTemplate:\n{\n'Nodule': '[ANSWER]',\n'Mass': '[ANSWER]',\n'Calcification': '[ANSWER]',\n'Asymmetry': '[ANSWER]',\n'Architectural Distortion': '[ANSWER]',\n'Skin Thickening': '[ANSWER]',\n'Lymph Node Enlargement': '[ANSWER]',\n'Intramammary Lymph Node': '[ANSWER]',\n'Nipple Retraction': '[ANSWER]',\n'Skin Retraction': '[ANSWER]',\n'Trabecular Thickening': '[ANSWER]',\n}\n\nReport:\nRight Breast:  Status postoperative showing operative scar with surgical clips and lower inner architectural distorton and nodular increased density associated with overlying 

 99%|█████████▉| 162/163 [6:05:46<02:16, 136.84s/it]

["As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \n\nRequirements:\n1. Provide the lesion's location as 'right', 'left' or 'both'(i.e, right and left) if the lesion is explicitly mentioned as present.\n2. Provide 'obscure' if the information is incomplete.\n3. Default to 'x' if the finding is not mentioned.\n\nTemplate:\n{\n'Nodule': '[ANSWER]',\n'Mass': '[ANSWER]',\n'Calcification': '[ANSWER]',\n'Asymmetry': '[ANSWER]',\n'Architectural Distortion': '[ANSWER]',\n'Skin Thickening': '[ANSWER]',\n'Lymph Node Enlargement': '[ANSWER]',\n'Intramammary Lymph Node': '[ANSWER]',\n'Nipple Retraction': '[ANSWER]',\n'Skin Retraction': '[ANSWER]',\n'Trabecular Thickening': '[ANSWER]',\n}\n\nReport:\nRight Breast:  No speculated mass lesions or suspicious microcalcifications.  Normal skin thickness and contour of breast.    Left Breast:  Upper outer quadrant rounded equal 

100%|██████████| 163/163 [6:08:03<00:00, 135.48s/it]

['As an AI trained in analyzing radiology reports, your task is to classify the presence or absence of specific findings in each report. Respond according to the given template. \n\nRequirements:\n1. Provide the lesion\'s location as \'right\', \'left\' or \'both\'(i.e, right and left) if the lesion is explicitly mentioned as present.\n2. Provide \'obscure\' if the information is incomplete.\n3. Default to \'x\' if the finding is not mentioned.\n\nTemplate:\n{\n\'Nodule\': \'[ANSWER]\',\n\'Mass\': \'[ANSWER]\',\n\'Calcification\': \'[ANSWER]\',\n\'Asymmetry\': \'[ANSWER]\',\n\'Architectural Distortion\': \'[ANSWER]\',\n\'Skin Thickening\': \'[ANSWER]\',\n\'Lymph Node Enlargement\': \'[ANSWER]\',\n\'Intramammary Lymph Node\': \'[ANSWER]\',\n\'Nipple Retraction\': \'[ANSWER]\',\n\'Skin Retraction\': \'[ANSWER]\',\n\'Trabecular Thickening\': \'[ANSWER]\',\n}\n\nReport:\nRight Breast:  \tUpper outer and retroareolar nodular asymmetrical increased density is seen.  \tUpper outer oval shaped

In [40]:
import os
inference_result_path = "/home/workspace/inference_result"
if USE_FINETUNED:
    file_name = checkpoint_name.split("/")[4] + f"-{DATASET}_{version}.xlsx"
    # file_name = checkpoint_name.split("/")[4] + "_prior10k.xlsx"
    # file_name = checkpoint_name.split("/")[4] + "-SNUH_test_v4.2.xlsx"
    # file_name = checkpoint_name.split("/")[4] + "-SNUH_training_v2.2.xlsx"
    # file_name = checkpoint_name.split("/")[4] + "-CDDCESM_fixed1.3.xlsx"
else:
    file_name = model_id.split('/')[-1] + f"_NSHOT{N_SHOT}-{DATASET}_{version}.xlsx"
pd.concat((data, pd.DataFrame(labels)), axis=1).to_excel(os.path.join(inference_result_path, file_name), index=False)
print("saved at ", inference_result_path+'/'+file_name)

saved at  /home/workspace/inference_result/llama-65b_NSHOT0-CDDCESM_fixed1.3.xlsx


In [59]:
labels

[{'Nodule': 'x',
  'Mass': 'x',
  'Calcification': 'x',
  'Asymmetry': 'x',
  'Architectural Distortion': 'l',
  'Skin Thickening': 'x',
  'Lymph Node Enlargement': 'x',
  'Intramammary Lymph Node': 'x',
  'Nipple Retraction': 'x',
  'Skin Retraction': 'x',
  'Trabecular Thickening': 'x'},
 {'Nodule': 'o',
  'Mass': 'b',
  'Calcification': 'r',
  'Asymmetry': 'o',
  'Architectural Distortion': 'o',
  'Skin Thickening': 'o',
  'Lymph Node Enlargement': 'o',
  'Intramammary Lymph Node': 'o',
  'Nipple Retraction': 'o',
  'Skin Retraction': 'o',
  'Trabecular Thickening': 'o'},
 {'Nodule': 'l',
  'Mass': 'l',
  'Calcification': 'x',
  'Asymmetry': 'x',
  'Architectural Distortion': 'x',
  'Skin Thickening': 'x',
  'Lymph Node Enlargement': 'x',
  'Intramammary Lymph Node': 'x',
  'Nipple Retraction': 'x',
  'Skin Retraction': 'x',
  'Trabecular Thickening': 'x'},
 {'Nodule': 'o',
  'Mass': 'o',
  'Calcification': 'o',
  'Asymmetry': 'o',
  'Architectural Distortion': 'o',
  'Skin Thickeni